# Generating the graph isomorphism dataset

## Setup

In [16]:
SAVE_DATA = False
LOAD_DATA = True

GENERATION_TEST_DATA_FILE = "data/generation_test_data.pkl"

In [15]:
from collections import Counter, defaultdict
from hashlib import blake2b
import pickle

import networkx as nx
from networkx import weisfeiler_lehman_graph_hash, erdos_renyi_graph

from sklearn.model_selection import ParameterGrid

from tqdm import tqdm

from rich.console import Console
from rich.table import Table

import plotly.graph_objs as go

## WL score

In [2]:
# Adapted from https://github.com/networkx/networkx/blob/main/networkx/algorithms/graph_hashing.py

def hash_label(label, digest_size):
    return blake2b(label.encode("ascii"), digest_size=digest_size).hexdigest()


def init_node_labels(G):
    # return {u: str(deg) for u, deg in G.degree()}
    return {u: "0" for u in G.nodes()}


def neighborhood_aggregate(G, node, node_labels):
    """
    Compute new labels for given node by aggregating
    the labels of each node's neighbors.
    """
    label_list = []
    for nbr in G.neighbors(node):
        label_list.append(node_labels[nbr])
    return node_labels[node] + "".join(sorted(label_list))


def weisfeiler_lehman_graph_score(
    G_1, G_2, max_iterations=5, digest_size=16
):
    
    if G_1.number_of_nodes() != G_2.number_of_nodes():
        return 0

    def weisfeiler_lehman_step(G, labels, edge_attr=None):
        """
        Apply neighborhood aggregation to each node
        in the graph.
        Computes a dictionary with labels for each node.
        """
        new_labels = {}
        for node in G.nodes():
            label = neighborhood_aggregate(G, node, labels)
            new_labels[node] = hash_label(label, digest_size)
        return new_labels
    
    G = [G_1, G_2]

    # set initial node labels
    node_labels = [init_node_labels(G[i]) for i in range(2)]

    subgraph_hash_counts = [[] for _ in range(2)]
    for iteration in range(max_iterations):
        graph_hash = [[] for _ in range(2)]
        for i in range(2):
            node_labels[i] = weisfeiler_lehman_step(G[i], node_labels[i])
            counter = Counter(node_labels[i].values())
            subgraph_hash_counts[i].extend(sorted(counter.items(), key=lambda x: x[0]))
            graph_hash[i] = hash_label(str(tuple(subgraph_hash_counts[i])), digest_size)
        if graph_hash[0] != graph_hash[1]:
            return iteration + 1
        
    return None


## Testing generation

In [5]:
num_pairs = int(1e6)
parameter_grid = {
    "edge_prob": [0.2, 0.4, 0.6, 0.8],
    "graph_order": [4, 8, 16, 32]
}
parameter_iter = ParameterGrid(parameter_grid)

In [6]:
if LOAD_DATA:
    with open(GENERATION_TEST_DATA_FILE, "rb") as f:
        results = pickle.load(f)
else:
    results = {}

    for params in parameter_iter:
        params_tuple = (params["edge_prob"], params["graph_order"])
        results[params_tuple] = {
            "score_2_count": 0,
            "score_gt_2_count": 0,
        }
        for i in tqdm(
            range(num_pairs), desc=f"{params_tuple}"
        ):
            G1 = erdos_renyi_graph(params["graph_order"], params["edge_prob"])
            G2 = erdos_renyi_graph(params["graph_order"], params["edge_prob"])
            score = weisfeiler_lehman_graph_score(G1, G2)
            if score == 2:
                results[params_tuple]["score_2_count"] += 1
            elif score is not None and score > 2:
                results[params_tuple][
                    "score_gt_2_count"
                ] += 1
        print(results[params_tuple])

(0.2, 4): 100%|██████████| 1000000/1000000 [00:38<00:00, 25814.62it/s]


{'score_2_count': 0, 'score_gt_2_count': 0}


(0.2, 8): 100%|██████████| 1000000/1000000 [00:42<00:00, 23636.42it/s]


{'score_2_count': 9572, 'score_gt_2_count': 87}


(0.2, 16): 100%|██████████| 1000000/1000000 [01:39<00:00, 10083.84it/s]


...

{'score_2_count': 9795, 'score_gt_2_count': 102}


(0.8, 16): 100%|██████████| 1000000/1000000 [02:22<00:00, 7001.83it/s]


{'score_2_count': 306, 'score_gt_2_count': 0}


(0.8, 32): 100%|██████████| 1000000/1000000 [07:28<00:00, 2228.82it/s]

{'score_2_count': 0, 'score_gt_2_count': 0}


In [18]:
if SAVE_DATA:
    with open(GENERATION_TEST_DATA_FILE, "wb") as f:
        pickle.dump(results, f)

In [9]:
table_2 = Table(title="Number of score 2 pairs")
table_gt_2 = Table(title="Number of score > 2 pairs")

table_2.add_column("Edge prob", justify="right")
table_gt_2.add_column("Edge prob", justify="right")
for graph_order in parameter_grid["graph_order"]:
    table_2.add_column(f"Order {graph_order}", justify="right")
    table_gt_2.add_column(f"Order {graph_order}", justify="right")

for edge_prob in parameter_grid["edge_prob"]:
    table_2.add_row(
        str(edge_prob),
        *[
            str(results[(edge_prob, graph_order)]["score_2_count"])
            for graph_order in parameter_grid["graph_order"]
        ],
    )
    table_gt_2.add_row(
        str(edge_prob),
        *[
            str(results[(edge_prob, graph_order)]["score_gt_2_count"])
            for graph_order in parameter_grid["graph_order"]
        ],
    )

console = Console()
console.print(table_2)
console.print(table_gt_2)

                Number of score 2 pairs                
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Edge prob ┃ Order 4 ┃ Order 8 ┃ Order 16 ┃ Order 32 ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│       0.2 │       0 │    9572 │      289 │        0 │
│       0.4 │       0 │    6551 │       46 │        0 │
│       0.6 │       0 │    6418 │       49 │        0 │
│       0.8 │       0 │    9795 │      306 │        0 │
└───────────┴─────────┴─────────┴──────────┴──────────┘

               Number of score > 2 pairs               
┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━┓
┃ Edge prob ┃ Order 4 ┃ Order 8 ┃ Order 16 ┃ Order 32 ┃
┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━┩
│       0.2 │       0 │      87 │        0 │        0 │
│       0.4 │       0 │      96 │        0 │        0 │
│       0.6 │       0 │     126 │        0 │        0 │
│       0.8 │       0 │     102 │        0 │        0 │
└───────────┴─────────┴─────────┴──────────┴──────────┘

In [12]:
# extract data from results dictionary
edge_probs = parameter_grid["edge_prob"]
graph_orders = parameter_grid["graph_order"]
score_2_counts = [[results[(edge_prob, graph_order)]["score_2_count"] for graph_order in graph_orders] for edge_prob in edge_probs]

# create heatmap
heatmap = go.Heatmap(
    z=score_2_counts,
    x=graph_orders,
    y=edge_probs,
    colorscale='Viridis'
)

# create layout
layout = go.Layout(
    title='Score 2 counts',
    xaxis=dict(title='Graph order'),
    yaxis=dict(title='Edge probability')
)

# create figure
fig = go.Figure(data=[heatmap], layout=layout)

# show figure
fig.show()

In [13]:
# extract data from results dictionary
edge_probs = parameter_grid["edge_prob"]
graph_orders = parameter_grid["graph_order"]
score_gt_2_counts = [[results[(edge_prob, graph_order)]["score_gt_2_count"] for graph_order in graph_orders] for edge_prob in edge_probs]

# create heatmap
heatmap = go.Heatmap(
    z=score_gt_2_counts,
    x=graph_orders,
    y=edge_probs,
    colorscale='Viridis'
)

# create layout
layout = go.Layout(
    title='Score > 2 counts',
    xaxis=dict(title='Graph order'),
    yaxis=dict(title='Edge probability')
)

# create figure
fig = go.Figure(data=[heatmap], layout=layout)

# show figure
fig.show()